<a href="https://colab.research.google.com/github/Echo9k/WebScrapping/blob/main/WebScrapping_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
#@markdown Install libraries
%%capture
!rm sample_data -r
!pip install fake-useragent
!pip install fastprogress
# !pip install w3lib
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
#@markdown Importing dependencies
from __future__ import print_function
import sys
import os
import re
import scipy
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# HTML
import requests
from bs4 import BeautifulSoup as bs
from requests.exceptions import HTTPError
from IPython.core.display import display, HTML
from urllib.parse import unquote
# import mechanize

# Selenium for JS support
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver

JavaScript
Try first soup.decode [BeautifulSoup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

If you need to try something else go with Phantom or headless_driver (slower).
```
def get_attribute(id, attribute):
    return headles_driver.find_element_by_id(id).get_attribute(attribute)
```



In [ ]:
#@markdown As: headles_driver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# #@markdown As: headles_driver
# headles_driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

#@markdown As: phantom_driver
# from selenium import webdriver
# !wget https://bitbucket.org/ariya/phantomjs/downloads/phantomjs-2.1.1-linux-x86_64.tar.bz2
# !tar xvjf phantomjs-2.1.1-linux-x86_64.tar.bz2
# !cp phantomjs-2.1.1-linux-x86_64/bin/phantomjs /usr/local/bin
# !ls -al
# phantom_driver = webdriver.PhantomJS()

In [ ]:
#@title Functions
## Lambdas
render = lambda html_contents: display(HTML(html_contents))

# @markdown * search_tag
def search_tag(tag, string):
    regex=r"(?:"+tag+"=)"
    if tag in string:
        split_1 = re.split(regex,string)[1].replace('%20', ' ')
        print(f"{tag}: found in text\n"
            f"contains:{split_1}")
# @markdown * regex_finder
def regex_finder(regex:str, text:str,*,
           look_before:int=10,
           look_ahead:int=250,
           extra_dots=1) -> str:
    """
    # RETURNS: group found, match
    """
    matches = re.finditer(regex, text, re.MULTILINE | re.IGNORECASE | re.UNICODE)
    
    for matchNum, match in enumerate(matches, start=1):
        print(f"Match: {matchNum} {match.group()}")
        # , match = match.group()
        
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            return match.group()[:-extra_dots], match.string[match.start(groupNum)-look_before:match.end(groupNum)+look_ahead]

# @markdown ### Tags:
# @markdown The RegEx strings are stored in the directory: _labels_ <br> <br>
regex_labels= {
    "Brand":r"(?:brand|brandname|vendor|manufacturer|product-brand)(?![&])(.)",
    "Manufacturer":r"(?:manufacturer|mfr|mfg|manufacturer logo|manufacturer name|label|producer|fabricante|fabrikant|Hersteller)(?![&])(.)",
    "Category":r"(?:category|categories|category path|breadcrumbs|breadcrum|crumb|navbar|Product Category)(?![&])(.)",
    "Model":r"(?:sku|model|model id|model no|item number|itemid|article no|product number|style number|product id|item code|mfr no|data-product)(?![&])(.)",
    "UPC":r'(?:"UPC"|"GTIN"|"EAN"|"upc"|"upccode"|"product_upc"|"product:upc"|"gtin"|"ean"|"barcode")',
    "Part":r"(?:PN|P/N|part no|part number|part|part #|mpn)(?![&g])(...)",
    "Color":r"(?:color|color_name|shade|finish|shade description)(?![&])(.)",
    "size":r"(?:selected size|available size|choose a size|product size|attribute pa size)(?![&])(.)",
    "list_price":r"(?:MSRP|MRP|Recommended Customer Price|USD MSRP|List Price|reseller price may vary)(?![&])(.)",
    "item_count":r"(?:count|pieces|ct|pc|combo|per pack|contains)(?![&])(.)",
    "IPQ":r"(?:packs|packs of|pk|package|combo|carton|carton pack)(?![&])(.)",
    "product_description":r"(?:Product Details|Specification|Tech specs|Technical specification|Details|see more features|Product Description|Description|About the product|ingredients|Where to use|How to use)(?![&])(.)"
    }

# Explore tags
One option we have to obtain the product detaill's from a URL would be using the labels corresponding and attribute to find their necesary value. We can use Regular Expressions (RegEx) to speed up this process.

In [ ]:
#@title Read URL
URL = "https://www.myblisskiss.com/dropper-squeeze-pen-combo-kit/?Pack=3%20Pack&Scent=Crisp" #@param {type:"string"}
show = False #@param {type:"boolean"}

# Retrieve URL
if len(URL)>0:
    response = requests.request('GET', URL,  headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs(response.text)
    pretty_soup = soup.prettify()
    if show:
        render(response.text)

In [ ]:
find = "UPC" #@param ["Brand", "Manufacturer", "Category", "Model", "UPC", "Part", "Color", "size", "list_price", "item_count", "IPQ", "product_description"]
look_ahead = 45 #@param {type:"slider", min:0, max:100, step:5}
try: display(regex_finder(regex_labels.get(find), soup.text, look_ahead=look_ahead))
except: "something happened"

In [ ]:
#@title By id and attribute

tag_type = "div" #@param {type:"string"}

#@markdown Attributes
attributes = {} #@param {type:"raw"}
_class = "woocommerce-product-details__short-description" #@param {type:"string"}
_id = "" #@param {type:"string"}
_itemprop = "" #@param {type:"string"}

if len(_class)>0: attributes.update({"class":_class})
if len(_id)>0: attributes.update({"id":_id})
if len(_itemprop)>0: attributes.update({"itemprop":_itemprop})

split_text = None #@param {type:"raw"}
strip_ = True #@param{type:"boolean"}

find_all = False #@param {type:"boolean"}
try:
    if find_all:
        test = soup.find_all(tag_type, attributes)
        formula = f"page.soup.find_all('{tag_type}', {attributes})"
        test = [t.text for t in test]
        print(f"len {len(test)}\n", test.get_text())
    else:
        test = soup.find(tag_type, attributes)
        formula = f"clean_it(page.soup.find('{tag_type}', {attributes}))"
        display(clean_it(test, split_text=split_text, strip_=strip_))
    display(f"{formula}")
except AttributeError: raise
except: print(page.url); raise

# Search
A different paradigm is to use a list of URL to retrive and save their soups at once, thus reducing server work and reducing processing time.

## Additional setup

In [ ]:
#@markdown Additional libraries
import json
import time
import unicodedata
from fake_useragent import UserAgent
from google.colab import files
from fastprogress import master_bar, progress_bar
# from w3lib.html import replace_entities
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from google.colab import data_table

In [ ]:
#@markdown Functions
# Labmdas
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

lap_time = lambda start_time: time.time() - start_time
def __variant_extractor(squeezed_url):
    try: return int(squeezed_url)
    except TypeError: return None
    except: return squeezed_url

def __url_info(url):
    return_ = {}
    try:
        url_info_asis = url.split('?',1)[1]
        details = url_info_asis.split('&')
        for i in details:
            split = i.split('=')
            return_.update({split[0]:split[1]})
    except IndexError: return_ = {"empty":None}
    except AttributeError: print(f"AttributeError {url}"); return_ = None
    except: raise
    return return_

def clean_it(text, *,form='NFKC', split_text = "\n" , strip_=True):
    if split_text is None: 
        text = text.get_text(strip=strip_)
    else:
        text= text.get_text(split_text, strip=strip_)
    if form is not None: return unicodedata.normalize(form, text)
    else: return text
# Retrieve data
def load_file(file_name, kwargs):
    def __read_csv(file_name, kwargs):
        try: return pd.read_csv(file_name, **kwargs)
        except: display("Invalid arguments (CSV)"); raise

    def __read_excel(file_name, kwargs):
        try: return pd.read_csv(file_name, **kwargs)
        except: display("Invalid kargs (Excel)"); raise

    # read
    if file_name.rsplit('.',1)[-1] in ['csv','text']:
        print('format: csv/text')
        return __read_csv(file_name, kwargs)
    elif file_name.rsplit('.',1)[-1] in ['xls','xlsx']:
        print('format: Excel')
        return __read_excel(file_name, kwargs)
    else: "unknown format"

## Column names
# all_columns=["ID","website","title","bread_crumb1","bread_crumb2","bread_crumb3","brand_name",
#              "manufacturer_name","model","upc","color_name","size_name",
#              "item_package_quantity","part_number","list_price","unit_count",
#              "product_description"]

usual_columns=["url","title","bread_crumb1","bread_crumb2","bread_crumb3",
            "brand_name", "manufacturer_name","model","upc",
             "color_name","size_name", "item_package_quantity","unit_count"]

In [ ]:
#@markdown Class: RetrivePage<br>
#@markdown If "Damnit, unicode" is guessing the encoder wrong
exclude_encodings = None #@param ["None"] {type:"raw", allow-input: true}



class RetrivePage():
    """This is a page class meant to retrive a page and return it's soup object"""

    def __init__(self, url, headless=False, phantom=False):
        def __variant_extractor(url):
            squeezed_url = np.squeeze(re.findall(r"=(.*)", x))
            try: return int(squeezed_url)
            except TypeError: return None
            except: return squeezed_url
        self.url = url
        self.response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        self.soup = bs(self.response.text, exclude_encodings=exclude_encodings)
        self.parsed_url = unquote(url)
        self.variant = __variant_extractor(self.url)
        self.metadata = {}
        self.variant_info = {}

## Workspace

In [ ]:
#@title Get the Websites

file_name = "/content/Pinkiou.csv" #@param {type:"string"}
kwargs = {"usecols": usual_columns} #@param ["{\"usecols\": usual_columns}", "{\"usecols\": [\"url\"]}", "{}"] {type:"raw", allow-input: true}
#@markdown Select **user agent**
# User agent to workaround sites that don't allow bots
header = "Chrome" #@param ["Chrome", "Edge", "Random", "None"]
usr_agent = UserAgent()

def user_agent_retrive(i):
    switcher={
            "Edge":usr_agent.edge,
            "Chrome":usr_agent.chrome,
            "Random":usr_agent.random
            }
    return switcher.get(i,None)
header = user_agent_retrive(header)
#@markdown _Looking for more options?_  [fake_useragent](https://pypi.python.org/pypi/fake-useragent)
def load_URLs(URL):
    r = requests.request('GET', URL, headers={'User-Agent': usr_agent})
    return bs(r.text), phantom_driver.get(URL), headles_driver.get(URL)

#@markdown Data stored as: data
data = load_file(file_name, kwargs)

## separating URLs
url_variant = data.url[data.url.apply(lambda x: "=" in x)]
not_variant = data.url[data.url.apply(lambda x: "=" not in x)]

print(f"Not variant URLs: {len(not_variant)}\n"
      f"URL with variant: {len(url_variant)}")

#@markdown Info from the URL: **url_info**
url_infos = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in data.url.apply(__url_info).items() ])).T
url_infos=url_infos.applymap(lambda x:unquote(str(x)))

In [ ]:
#@markdown ...loading<br>
#@markdown Create a new pd.Series to store the web scrapped RetrivePage objects.<br>
start_time = time.time()
pages = pd.Series([RetrivePage(url) for url in progress_bar(data.url)])
lap_time(start_time)

In [ ]:
#@title Find index of a given URL
_get_url = lambda x:x.url
urls = pages.apply(_get_url)
urls_dict = {url:i for i,url in enumerate(urls)}
url = "https://www.pinkiou.com/product/pinkiou-eyebrow-tattoo-needles-for-permanent-makeup-penmicroblading-pen/?size=tattoo%20needles&color=7%20Pin" #@param {type:"string"}
urls_dict.get(url)

In [ ]:
#@markdown Retrieve a page from it's index

index =    105#@param {type:"integer"}
page = pages[index]
soup = page.soup
url = page.url
variant = page.variant
example = {'index':index,
           'page':page,
           'soup':soup,
           'url':url,
           'variant':variant}
print(f"url: {page.url}")
show = False #@param {type:"boolean"}
if show:
    render(response.text)

## Test this first
These are functions that have worked well in the past.
Give them a try before experimenting something new.

In [ ]:
#@markdown #### Regex test*
pattern = '(\d+-)*\d+mm'
pattern = re.compile(pattern)

extracts = [re.search(pattern, str(page)) for page in data.title]
def _ex(e):
    try: return e.group()
    except: return None
pd.Series([_ex(e) for e in extracts]).to_csv("extracts.csv")

### From scripts
Often pages relay on a data structure for keeping the information of the different variants.

In [ ]:
#@title Simple script meta
attributes = {"data-desc":"seo-product", "type":"application/ld+json"} #@param{trype:"raw"}
key_name = "script_meta" #@param {type:"string"}
def __script_meta(page):
    script_meta = soup.find('script', attributes).text
    json_data = json.loads(script_meta)
    try: page.metadata.update({key_name:json_data})
    except: page.script_meta = None
    return page.metadata[key_name]
apply_to_all = True #@param{type:"boolean"}
if apply_to_all: pages.apply(__script_meta)
__script_meta(page)

In [ ]:
# @title Metadata
# @markdown This is useful when you have a tag such as **var meta =**
var_name = "var __st=" #@param {'type':'string'}
key_name = "__st" #@param {'type':'string'}
if key_name is None:
    key_name = var_name.strip().split()[1]
get_variant_info = False #@param {type:"boolean"}

print(f"key_name: {key_name}")
def __get_variant_info(page):
    # try:
    #     variants = page.metadata['product']['variants']
    #     return [i for i in variants if int(i['id']) == int(page.variant)][0]
    # except: None
    try: return [v for v in page.metadata[key_name]['product']['variants'] if int(v['id']) == int(page.variant)][0]
    except: None

def get_meta(page):
    soup = page.soup
    scripts = soup.find_all('script')
    # mets = ''
    for s in scripts:
        if var_name in s.text:          # find the script of interest
            # Exctract the json value of var_name
            meta = s.text
            meta = meta.split(var_name)[1].split('};')[0]+'}'
            # Load the json and make's sure the format is correct
            json_meta = json.loads(meta)
            page.metadata.update({key_name:json_meta}) # add metadata to the RetrivedPage
            if get_variant_info:
                page.variant_info.update({key_name:__get_variant_info(page)})
            return json_meta
json_meta = pages.apply(get_meta)
display(get_meta(page))

In [ ]:
#@title Script metadata
#@markdown For when the metadata comes from a script tag
identifier="bold-platform-data" #@param {type:"string"}
left_delimiter = '}' #@param {type:"string"}
script_idx  =  0#@param {'type':'integer'}
#@markdown Save as:
script_key = "script_meta" #@param {'type':'string'}
def __script_metadata(page):
    def __get_variant_info(page):
        try: return [v for v in page.metadata[script_key]['product']['variants'] if int(v['id']) == int(page.variant)][0]
        except: None
    
    def _clean_script(identifier, left_delimiter=None, script_idx=0):
        script = [i for i in page.soup.find_all('script') if identifier in str(i)][script_idx]
        script_clean = '{'+str(script).split('{',1)[1].split(left_delimiter,1)[0]+left_delimiter+'}}'
        try: return json.loads(script_clean)
        except json.JSONDecodeError:
            print(f"JSONDecodeError {page.url}")
            return script_clean

    
    try:
        script_metadata = _clean_script(identifier, left_delimiter, script_idx)
        page.metadata.update({script_key:script_metadata})
        try: page.variant_info.update({script_key:__get_variant_info(page)})
        except: pass
        return script_metadata
    except:
        return None
print(f"script_key: {script_key}")
script_metadatas = pages.apply(__script_metadata)
script_metadatas[index]

In [ ]:
#@title By attribute
#@markdown Find items by their attributes

tag_type = "div" #@param {type:"string"}

#@markdown Attributes
attributes = {} #@param {type:"raw"}
_class = "" #@param {type:"string"}
_id = "breadcrumbs" #@param {type:"string"}
_itemprop = "" #@param {type:"string"}

if len(_class)>0: attributes.update({"class":_class})
if len(_id)>0: attributes.update({"id":_id})
if len(_itemprop)>0: attributes.update({"itemprop":_itemprop})

split_text =  None#@param {type:"raw"}
strip_ = True #@param{type:"boolean"}

find_all = False #@param {type:"boolean"}
try:
    if find_all:
        test = soup.find_all(tag_type, attributes)
        formula = f"page.soup.find_all('{tag_type}', {attributes})"
        test = [t.text for t in test]
        print(f"len {len(test)}\n", test.get_text())
    else:
        test = soup.find(tag_type, attributes)
        formula = f"clean_it(page.soup.find('{tag_type}', {attributes}))"
        display(clean_it(test, split_text=split_text, strip_=strip_))
    display(f"{formula}")
except AttributeError: raise
except: print(page.url); raise

### Useful functions to start with

In [ ]:
#@title titles
def _title(page):
    try: page.title = clean_it(page.soup.find('h1', {'class': 'product_title entry-title'}))
    except: page.title = None
    return page.title

titles = pages.apply(__title)
print(titles[index])
titles

In [ ]:
#@title Crumbs
def __get_crumbs(page):
    result = {"crumb_0":None,"crumb_1":None,"crumb_2":None}
    try:
        # Just change this one
        crumbs =  [i.strip() for i in clean_it(page.soup.find('div', {'id': 'breadcrumbs'})).split('/')]
        if crumbs[0] != page.title:
            try: result["crumb_0"]=crumbs[0]
            except: pass
            if crumbs[1] != page.title:
                try: result["crumb_1"]=crumbs[1]
                except: pass
                if crumbs[2] != page.title:
                    try: result["crumb_2"]=crumbs[2]
                    except: pass
    except AttributeError: print(page.url)
    except: raise
    finally: return result
crumbs = pages.apply(__get_crumbs)

bread_crumb1 = pd.Series([i['crumb_0'] for i in crumbs])
bread_crumb2 = pd.Series([i['crumb_1'] for i in crumbs])
bread_crumb3 = pd.Series([i['crumb_2'] for i in crumbs])

display(crumbs[index])

print("unique values:\n"
    f"• bread_crumb1: {bread_crumb1.nunique()}\n"
    f"• bread_crumb2: {bread_crumb2.nunique()}\n"
    f"• bread_crumb3: {bread_crumb3.nunique()}\n"
    )
display(crumbs.sample(10))

#  Experiments

### Work in progress experiments

# Export

In [ ]:
#@markdown Result
result = {"url":urls}
result.update({
    "title":titles,
    "bread_crumb1":breadcrumb1,
    "bread_crumb2":breadcrumb2,
    "bread_crumb3":breadcrumb3,
    # "brand_name":brands,
    # 'manufacturer_name':manufacturer_names,
    # "product_description":descriptions,
    # "model":skus,
    # "upc":barcodes,
    # "part_number":MPNs,
    # "color_name":colors,
    # "size_name":sizes,
    # "item_package_quantity":IPQs
    # "unit_count":unit_counts

    # Temporarl keys
    # "sku_upc":
    })

In [ ]:
#@markdown Progress
show_progress = True #@param {type:"boolean"}
df_result = pd.DataFrame(result)
if show_progress:
    display(data_table.DataTable(df_result))

In [ ]:

df_result = pd.DataFrame(result)
export = "reviewed" #@param ["reviewed", "joined"]
fill_na_with = "___BLANK_CELL___" #@param  {type:"string"}
auto_download = True #@param {type:"boolean"}

_export_name = file_name.split('.')[0] + f"_{export}.xlsx"
_result_column_set = set(df_result.columns)
_data_column_set = set(data.columns)
_intersection = _data_column_set.intersection(_result_column_set)
_intersection.discard('url') # inplace opperation
print("Intersection", _intersection)

df_join = data.drop(columns=_intersection)
df_join = pd.concat([df_join.set_index('url'), df_result.set_index('url')],axis=1,)\
                .fillna(fill_na_with).replace('',fill_na_with)

_df_join_columns=set(df_join.columns)
_df_join_columns.discard('url')

if export == "joined":
    try:
        df_join = df_join.reindex(columns=list(_df_join_columns))
        print(f"df_join \t[reindexed]\t{time.time()}")
        if auto_download: files.download(_export_name)
    except:
        print(f"df_join \t[couldn't reindex]\t{time.time()}")
        display(_df_join_columns.symmetric_difference(_data_column_set))
    df_join.to_excel(_export_name)
    if auto_download: files.download(_export_name)
else:
    df_result.fillna(fill_na_with)\
            .replace('',fill_na_with)\
            .to_excel(_export_name)
    if auto_download: files.download(_export_name)
    print(f"df_result\t{time.time()}")